### INSTALACION DE LOS PAQUETES NECESARIOS

In [1]:
! pip install -U spacy
! python -m spacy download en
! pip install spacymoji
! pip install emoji
! pip install gensim
! pip install sklearn

import pandas as pd
import spacy
import emoji
import numpy as np

from sklearn.cluster import DBSCAN
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from gensim.test.utils import get_tmpfile


     --------------------------------------- 12.1/12.1 MB 36.4 MB/s eta 0:00:00
     ---------------------------------------- 48.9/48.9 kB ? eta 0:00:00
     ---------------------------------------- 181.6/181.6 kB ? eta 0:00:00
     --------------------------------------- 15.8/15.8 MB 36.4 MB/s eta 0:00:00
     ---------------------------------------- 57.0/57.0 kB ? eta 0:00:00
     ---------------------------------------- 50.0/50.0 kB ? eta 0:00:00
     ---------------------------------------- 395.8/395.8 kB ? eta 0:00:00
     ---------------------------------------- 78.3/78.3 kB ? eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 31.3 MB/s eta 0:00:00
     -------------------------------------- 122.2/122.2 kB 7.5 MB/s eta 0:00:00
     ------------------------------------- 481.9/481.9 kB 29.5 MB/s eta 0:00:00
     ---------------------------------------- 45.9/45.9 kB ? eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 42.1 MB/s eta 0:00:00
     --


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     --------------------------------------- 12.8/12.8 MB 21.1 MB/s eta 0:00:00
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use
the full pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 358.9/358.9 kB 3.7 MB/s eta 0:00:00


...


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     --------------------------------------- 24.0/24.0 MB 31.2 MB/s eta 0:00:00
     --------------------------------------- 44.1/44.1 MB 34.4 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip
  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [18 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is availab

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


ModuleNotFoundError: No module named 'pandas'

### CARGAR FICHERO CSV CON LOS DATOS

In [ ]:
data = pd.read_csv('../Datasets/Suicide_Detection.csv')

### SELECCIONAR LA COLUMNA DE TEXTOS (DISCRIMINAR LA CLASE)

In [ ]:
textos = []
for instancia in data.values:
    textos.append(instancia[1])

### LIMPIEZA Y FORMARTEO DE LOS DATOS

In [ ]:
nlp = spacy.load("en_core_web_sm") # Cargar modelo
nlp.add_pipe("emoji", first=True)

textos_tokenizados =[]
for t in textos:
    t= emoji.demojize(t) # Emojis a texto
    doc=nlp(t)
    lexical_tokens = [token.lemma_.lower() for token in doc if len(token.text) > 3 and token.is_alpha]
    textos_tokenizados.append(lexical_tokens)


### VECTORIZACION DE LOS DATOS (DOC2VEC)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(textos_tokenizados)]
model = Doc2Vec(documents, vector_size=150, window=2, dm=1, epochs=100, workers=4)
# DM siempre a 1 para que sea distributed memory (que tenga en cuenta el contexto)
# Window se puede cambiar pero tener en cuenta que cuanto mayor sea mas coste genera
# Negative: siempre a 0 para que aprenda que palabras son ruido
#Los que se pueden probar para parametro ekorketa:
    # VECTOR SIZE!!!! (tamaño del embedding: dimensiones)
    # Epochs: numero de iteraciones por el dataset
    # Alpha: learning rate inicial
    # Min alpha: El valor de LR con el que va acabar → Igual no poner?
    # Max_vocab_size: Limita el tamaño maximo de las palabras a tener en cuenta (borra las menos frecuentes)

model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

model.save(get_tmpfile("my_doc2vec_model"))

#vector = model.infer_vector(['wife', 'threaten', 'leave', 'wife', 'good', 'because', 'cheat', 'twice', 'lie', 'much', 'that', 'have', 'decide', 'refuse', 'back', 'day', 'begin', 'threaten', 'suicide', 'have', 'tirelessly', 'spend', 'these', 'paat', 'day', 'talk', 'keep'])

#similar_documents = model.dv.most_similar(vector) # Deberia salir 0 porque es el documento del que se han sacado todas las palabras

### DBSCAN (PREDETERMINADO)

In [ ]:
document_vectors = [model.infer_vector(doc) for doc in textos_tokenizados]

# Aplicar DBSCAN a los vectores de documentos
dbscan = DBSCAN(eps=2, min_samples=2, leaf_size=5)  # Ajusta los parámetros según tu caso
labels = dbscan.fit_predict(np.array(document_vectors))

# Los resultados del clustering están en 'labels'
print("Etiquetas de clusters:", labels)